# Document Embedding Basic Usage Example

Here, we provide a basic look into how the API can be used.

In [1]:
import os
os.chdir('..')

We use a sample scikit-learn dataset to test the embeddings.

In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
documents = newsgroups_train.data[:20]

In general, you can use `Embedding` models that take in raw text to convert them into real valued vectors. You can then follow them up with `Transformation` models that take in and output real valued vectors.

For example, we can replicate a TfIdf to NMF featurization in the following manner:

In [3]:
from textwiser import TextWiser, Embedding, Transformation

emb = TextWiser(Embedding.TfIdf(min_df=5), Transformation.NMF(n_components=10))
vecs = emb.fit_transform(documents)
vecs

array([[3.01101089e-01, 0.00000000e+00, 2.18839020e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.21987301e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [2.84159798e-02, 1.05698027e-01, 0.00000000e+00, 5.06473854e-02,
        0.00000000e+00, 4.90010940e-02, 6.90960288e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.12491953e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.87781453e-02,
        6.97313622e-02, 0.00000000e+00, 6.23105109e-01, 2.10211650e-01,
        0.00000000e+00, 6.22027293e-02],
       [0.00000000e+00, 0.00000000e+00, 7.93895423e-01, 8.12194590e-03,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.90453234e-03, 0.00000000e+00],
       [7.57250178e-04, 5.78031968e-03, 0.00000000e+00, 3.01019084e-02,
   

We can just as easily try out a pooling of word embeddings:

In [4]:
from textwiser import TextWiser, Embedding, PoolOptions, Transformation, WordOptions

emb = TextWiser(Embedding.Word(word_option=WordOptions.word2vec, pretrained='en'), Transformation.Pool(pool_option=PoolOptions.max))
vecs = emb.fit_transform(documents)
vecs

array([[0.1969, 0.2157, 0.2076, ..., 0.4019, 0.2006, 0.2305],
       [0.2327, 0.3131, 0.3033, ..., 0.3267, 0.2783, 0.2475],
       [0.0992, 0.1078, 0.207 , ..., 0.2473, 0.1487, 0.1541],
       ...,
       [0.2538, 0.2691, 0.1851, ..., 0.3561, 0.3472, 0.1758],
       [0.2038, 0.1847, 0.4164, ..., 0.3179, 0.24  , 0.2351],
       [0.2573, 0.2906, 0.2647, ..., 0.4127, 0.205 , 0.2459]],
      dtype=float32)

## Schema

We also provide a convenient schema for mixing and matching different word and document embeddings. This makes it easy to try out different embedding types rapidly.

There are two main operations: `transform` and `concat`.

The `transform` operation defines a list of operations. The first of these operations should be an `Embedding`, while the rest should be `Transformation`s. The idea is that the `Embeddings` have access to raw text and turn them into vectors, and therefore the following `Transformation`s need to operate on vectors. In PyTorch terms, this is equivalent to using `nn.Sequential`.

The `concat` operation defines a concatenation of multiple embedding vectors. This can be done both at word and sentence level. In PyTorch terms, this is equivalent to using `torch.cat`.

### Sample Schema

Below we outline a sample and presumably common use-case. At the root level, we have three different embeddings. The first two are different word embeddings, both are pooled (first using max pooling, second using mean pooling) to generate the sentence representations $s_1$ and $s_2$. The third is a tf-idf embedding of the document followed by a reduction of its dimensionality to 30 using NMF, generating sentence representation $s_3$. These representations are concatenated to $s_{123}$, and are fed into a final SVD transformation that brings the sentence vector back down to a manageable level ($s$).

In [5]:
from textwiser import TextWiser, Embedding

doc_embeddings_schema = {
    'transform': [
        {
            'concat': [
                {
                    'transform': [
                        ('word2vec', {'pretrained': 'en'}),
                        'pool'
                    ]
                },
                {
                    'transform': [
                        ('flair', {'pretrained': 'news-forward-fast'}),
                        ('pool', {'pool_option': 'mean'})
                    ]
                },
                {
                    'transform': [
                        'tfidf',
                        ('nmf', { 'n_components': 10 })
                    ]
                }
            ]
        },
        'svd'
    ]
}

doc_embeddings = TextWiser(Embedding.Compound(schema=doc_embeddings_schema))
doc_embeddings

TextWiser(
  (_imp): _Sequential(
    (0): _CompoundEmbeddings()
  )
)

Once the embeddings object is initialized, we can feed in a list of text documents and get the relevant output.

In [6]:
doc_embeddings.fit(documents)

TextWiser(
  (_imp): _Sequential(
    (0): _CompoundEmbeddings(
      (model): _Sequential(
        (0): _Concat(
          (embeddings): ModuleList(
            (0): _Sequential(
              (0): _WordEmbeddings(
                (model): Embedding(1000001, 300)
              )
              (1): _PoolTransformation()
            )
            (1): _Sequential(
              (0): _WordEmbeddings(
                (model): FlairEmbeddings(
                  (lm): LanguageModel(
                    (drop): Dropout(p=0.25, inplace=False)
                    (encoder): Embedding(275, 100)
                    (rnn): LSTM(100, 1024)
                    (decoder): Linear(in_features=1024, out_features=275, bias=True)
                  )
                )
              )
              (1): _PoolTransformation()
            )
            (2): _Sequential(
              (0): _TfIdfEmbeddings()
              (1): _NMFTransformation()
            )
          )
        )
        (1): _SVDTransform

In [7]:
%%time

emb = doc_embeddings.transform(documents)
emb

CPU times: user 2min 5s, sys: 1.34 s, total: 2min 7s
Wall time: 32.4 s


array([[-5.10063171e+00,  2.22280920e-02,  4.14250344e-01,
        -2.65658572e-02,  5.45007050e-01, -3.32126498e-01,
         3.22970182e-01, -7.25271821e-01,  9.28251565e-01,
        -2.71624506e-01],
       [-5.28810358e+00, -7.73512125e-02, -2.03879401e-02,
         1.06850453e-01, -1.51873410e-01,  2.99940258e-01,
        -3.02130491e-01, -1.00054994e-01,  1.08959593e-01,
        -1.06767222e-01],
       [-4.51112938e+00,  4.14192319e-01, -2.05684334e-01,
        -8.41357410e-01,  2.04449058e-01,  5.57696484e-02,
         3.18822205e-01,  3.17744553e-01,  1.21957198e-01,
         2.29578197e-01],
       [-5.08173132e+00, -4.44692105e-01,  2.89135247e-01,
        -3.85956839e-02,  2.49053538e-02,  1.28525406e-01,
         4.20099646e-02, -3.15635055e-01, -2.16314986e-01,
        -2.93152303e-01],
       [-4.58816624e+00,  6.82748616e-01, -7.39156455e-02,
        -4.03802037e-01, -3.86146009e-01, -1.87553063e-01,
        -7.91937590e-01, -4.82498974e-01, -4.54805315e-01,
        -5.

You can also specify the same schema in a json file.

In [8]:
import json

doc_embeddings = TextWiser(Embedding.Compound(schema='notebooks/schema.json'))
doc_embeddings

TextWiser(
  (_imp): _Sequential(
    (0): _CompoundEmbeddings()
  )
)

In [9]:
doc_embeddings.fit_transform(documents)

array([[-5.10063171e+00,  2.22280920e-02,  4.14250344e-01,
        -2.65658572e-02,  5.45007050e-01, -3.32126498e-01,
         3.22970182e-01, -7.25271821e-01,  9.28251565e-01,
        -2.71624506e-01],
       [-5.28810358e+00, -7.73512125e-02, -2.03879401e-02,
         1.06850453e-01, -1.51873410e-01,  2.99940258e-01,
        -3.02130491e-01, -1.00054994e-01,  1.08959593e-01,
        -1.06767222e-01],
       [-4.51112938e+00,  4.14192319e-01, -2.05684334e-01,
        -8.41357410e-01,  2.04449058e-01,  5.57696484e-02,
         3.18822205e-01,  3.17744553e-01,  1.21957198e-01,
         2.29578197e-01],
       [-5.08173132e+00, -4.44692105e-01,  2.89135247e-01,
        -3.85956839e-02,  2.49053538e-02,  1.28525406e-01,
         4.20099646e-02, -3.15635055e-01, -2.16314986e-01,
        -2.93152303e-01],
       [-4.58816624e+00,  6.82748616e-01, -7.39156455e-02,
        -4.03802037e-01, -3.86146009e-01, -1.87553063e-01,
        -7.91937590e-01, -4.82498974e-01, -4.54805315e-01,
        -5.